In [1]:
%autosave 120

In [2]:
import pandas as pd
import spacy
#pip install nltk
from nltk.tokenize import word_tokenize
import numpy as np
import en_core_web_sm
import re


In [3]:
import nltk
#nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiyanzhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data

In [14]:
train = pd.read_csv(r'data/train.csv')
X_train = train.comment_text

In [15]:
X_train.shape

(1804874,)

# Pre-process the text

In [6]:
# retrieved from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python


def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


In [ ]:
def preprocess(corpus):
    '''This pre-processing function lemmatizes words, makes a commment case-insensitive, removes punctuation,
    removes stop words and removes single characters. '''
    
    # convert df to list; *corpus* is of a form of a list of str
    original_indices = corpus.index
    corpus = corpus.to_list()
    
    # initialize lemmatizer and a list of stopwords
    lemmatizer = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    
    for i in range(len(corpus)):
        text = corpus[i] # one comment
        
        # convert text to lower case
        text = text.lower()
        
        # remove emoji
        text = deEmojify(text)
        
        # remove url links
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"www\S+", "", text)
        
        # remove punctuations and numbers but save the "'"
        symbols = "—…,!\"#$%&()*+-./:;<=>?@[\]^_`{|}~0123456789"
        for sym in symbols:
            text = np.char.replace(text, sym, ' ') # replace symbols by one space

        # tokenize text
        text_tokens = word_tokenize(str(text))

        # spaCy Lemmatization
        doc = lemmatizer(" ".join([tk for tk in text_tokens]))
        #print(doc)
        corpus[i] = " ".join([token.lemma_ for token in doc]) # Extract the lemma for each token and join
    return pd.Series(corpus, index=original_indices)